In [10]:
import pandas as pd
import numpy as np
import pycountry
import countrydict
import province_to_regions

In [11]:
df = pd.read_excel("emdat_public_2023_02_14_query_uid-AJtVn9.xlsx",skiprows = 6)
df = df[df['Insured Damages (\'000 US$)'].isna() == False]


In [12]:
#A semi-automated way to cluster some rows when the country spans multiple regions
def get_region(iso,location):
    #If the country only spans one region, we can get it directly, otherwise we need to look at the provinces/states
    regions = countrydict.countries_to_regions[iso]
    if(isinstance(regions,str)):
        return regions
    else:
        dict = province_to_regions.provinces_to_region[iso]
        for province,region in dict.items():
            if(location.find(province) != -1):
                return region
            #The china dictionary is structured differently since the majority of provinces are in the EAS region. We only need to consider tibet and xinjiang seperately
            if(iso == "CHN"):
                return "EAS"
        return "None"
    
df['Location'].fillna("",inplace=True)
df['Region'] = df.apply(lambda row: get_region(row.ISO,row.Location),axis = 1)

In [13]:
#Cluster 0 corresponds to losses that need to be considered or manually
df['Cluster'] = df['Region'].map(lambda x: countrydict.regions_to_cluster[x] if (isinstance(x,str)) else 0)

In [14]:
df[df.Cluster == 0].to_excel("manual_clusters.xlsx")
#About 50 events need to be considered manually now